In [1]:
import pandas as pd
import numpy as np
import cvxpy as cvx
import random
import time


from scipy.linalg import hadamard

import matplotlib.pyplot as plt

from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold

from scipy import stats
from matplotlib import pyplot
from scipy.stats import spearmanr

import warnings
warnings.filterwarnings("ignore")


import matplotlib
matplotlib.rc('text.latex', preamble=r'\usepackage{amsmath}')

import matplotlib.font_manager as fm


import seaborn as sns

table = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/will-luddington-v4/A5.pkl")
tableg = table.T
B = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/will-luddington-v4/B5.pkl")

Bdf = pd.DataFrame(B)

species_ones = [] 
for i in range(5):
    species = i
    species_ones.append(list(tableg[tableg[species]==1].index))
    
    
species_zeros = []
for i in range(5):
    species = i
    species_zeros.append(list(tableg[tableg[species]==0].index))                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

sp_list_provided = ['LP', 'LB', 'AP', 'AT', 'AO']

#### get all data -- design matrix, steady state abundances, and list of wells present


sst = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/will-luddington-v5/Gould-etal-5species-relative-abundance-Oct22.pkl")
sstv3 = sst.copy()


from sklearn.metrics import r2_score


import matplotlib.patches as mpatches

from sklearn.ensemble import RandomForestRegressor

In [2]:
csdf = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/will-luddington-v5/rel_CS_WH.pkl")

cv_splitsl = [3] 
max_depthl = [2,5,10,15]
ntrees = [100,200,300,400,500,1000]
max_featuresl = ["auto","sqrt","log2"]

In [3]:
trdatadf =  pd.read_pickle("~/compressed_sensingv1/realdatasets/will-luddington-v6/traindataRELwill.pkl")
trdatadf.head(3)

tedatadf =  pd.read_pickle("~/compressed_sensingv1/realdatasets/will-luddington-v6/testdataRELwill.pkl")
tedatadf.head(3)

jd = pd.concat([trdatadf.T,tedatadf.T]).T
df = jd
df = df.loc[:,~df.columns.duplicated()].copy()

In [ ]:
rfper = [] 
startt = time.time()
for cv_splits in cv_splitsl: 
    for species in range(5):
            spapples = df[df["species"]==species]
            datasplits = spapples[spapples["k-fold"]==cv_splits][:cv_splits]
            datasplits.index = np.arange(0,cv_splits,1)
            abunvec = pd.DataFrame(sstv3[species][species_ones[species]].values)
            data_present = abunvec.index 
            Xraw = tableg.T[species_ones[species]].T
            Xraw.index = abunvec.index
            xs = list(data_present)
            random.shuffle(xs)
            data_present = np.array(xs)
            for md in max_depthl:
                for n_est in ntrees:
                    for mfeatures in max_featuresl:
                        for folds in range(cv_splits):
                            rinew = datasplits["train data"][folds]
                            rileft = datasplits["test data"][folds]
                            y3 = abunvec.T[rinew].T[0].values   
                            x3 = Xraw.T[rinew].T
                            regr = RandomForestRegressor(max_depth=md,n_estimators=n_est,max_features=mfeatures)
                            regr.fit(x3,y3)
                            ypred = regr.predict(Xraw.T[rileft].T)
                            rfper.append([species,md,n_est,rinew,rileft,ypred,cv_splits,mfeatures])
print(time.time()-startt)

In [ ]:
pd.DataFrame(rfper)

In [ ]:
rdf = pd.DataFrame(rfper,columns = ["species","max_depth","n_trees","Xtrain","Xtest","ypred","k-fold","max_features"])

In [ ]:
rdf.to_pickle("RF-5sp-will-REL.pkl")